In [1]:
!pip install youtube_transcript_api

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon May  4 11:12:28 2020

@author: Siebe Albers
"""
## edited by Ivette Bonestroo 04-26-2021 

#======================================================================== #
'load a df with video ids (which will be used for the youtube api to download the transcripts: and later on for extracting the labels'  
import pandas as pd                        
dataset = pd.read_csv('autumn 2021.csv') 
#with open('IdList_selfWachtedYoutubeVids.txt', encoding="utf-8") as f:
 #   idList2 = f.readlines() #txt file with the ids:
idList = dataset['ID'].tolist()
print(idList)
print(len(idList))
#alternatively:
#dic = dict_oldDf # a dictionary where the keys correspond the the youtubeIDs

['04wPQu3BC2k', 'WAeBPskvP4Y', 'A8e9gvrZEps', 'HJG96xfyOk4', 'V0NGZcorqZ4', 'kj5ZEtCcnQk', 'VzM5X4udX1E', 'eadUggsvmNo', 'uYF3KLv0_3A', 'H4iwjnbHkCw', 'x5p18S_S8N0', 'cMLaJsSrZuQ', 'gslI7cu1C4M', 'tdcDGKXGTsg', '24d9BFtXPbc', '61gIgxFXy9U', 'QE3hR3rMHOI', '0uBYomUSOoM', 'pMy2Lf40Xts', 'sIBR4ZSnSFQ', '8eqtEw2AMTw', 'KvkbKVwYT1A', 'iymP2jq6nFs', 'dHQQm_sJHDA', 'bhMEqrpf5xM', 'cuZ6oA9MqhE', 'I8h7unfqCLw', 'uMIG00gPyns', 'yk-z-XHvNqM']
29


In [3]:
#======================================================================== #
' downloading the transcripts by their ids                           '
#======================================================================== #
from youtube_transcript_api import YouTubeTranscriptApi
import time # just to record how long it takes to download the transcripts
STARTTIME = time.time() #plus counting the time,
Transcripts_w_timestamps1 =YouTubeTranscriptApi.get_transcripts(video_ids=idList,continue_after_error=True)

Transcripts_w_timestamps = Transcripts_w_timestamps1[0]

print('time it took:', time.time() - STARTTIME)

print( 'len trans', len(Transcripts_w_timestamps)) # see how many could be downloaded


# =============================================================================
# transcripts that were unable to be extraced:
# =============================================================================
#ids_thatcouldnotbedownloaded = list( set_originalId - set(downloadedtransIds )
#print( 'len downloaded trans:',ids_thatcouldnotbedownloaded)

not_extracted = Transcripts_w_timestamps1[1]
Transcripts_w_timestamps2 =YouTubeTranscriptApi.get_transcripts(video_ids=not_extracted,continue_after_error=True, cookies = 'cookies.txt') 
## cookies txt is created based upon the cookies of these videos: 
##['areIv5h_vss', '4FN12sqoC4Y', 'UX5OwBnhksY', 'JYxJkq5B6ec', 'fMxFSfIV3Dc']
## cookies.txt add-on for chrome is used to extract the txt. 
##The last two rows of the last four generated txtfiles are manually added to the first txtfile: 'cookies.txt'.
Transcripts_w_timestamps2 = Transcripts_w_timestamps2[0]
print( 'len trans', len(Transcripts_w_timestamps2))

time it took: 13.939071416854858
len trans 27
len trans 0


In [4]:
# =============================================================================
# # creating a dict with transcripts, ψ Writing to string files to (re)create the transcripts
# =============================================================================
#create a list of video ids, serving as keys for next para
IDLIST = list(Transcripts_w_timestamps.keys())
IDLIST2 = list(Transcripts_w_timestamps2.keys())

trans_dic_fromApi = {}
for I in IDLIST:
    TRANS = ""
    trans_dic_fromApi[I] = None
    for J in Transcripts_w_timestamps[I]:
#        print(J['text'])
        TRANS += J['text']
        TRANS += " "
    trans_dic_fromApi[I] = TRANS

for I in IDLIST2:
    TRANS = ""
    trans_dic_fromApi[I] = None
    for J in Transcripts_w_timestamps2[I]:
#        print(J['text'])
        TRANS += J['text']
        TRANS += " "
    trans_dic_fromApi[I] = TRANS

In [5]:
#======================================================================== #
' Exporting to disk         '
#======================================================================== #
import json
with open('ourWatchedYoutubevidsTranscriptsψkeys.json', 'w') as fp:
    json.dump(trans_dic_fromApi, fp)





## =============================================================================
## #creating 1 dictionary for storing all meta Data of the transcripts
## =============================================================================
#a_meta_dict_transFromApi = { 'transcripts': {I: trans_dic_fromApi[I] for I in trans_dic_fromApi.keys() }, #look at the weird syntac I: ...[i] for i 
#         'lengths': {I: len(trans_dic_fromApi[I].split()) for I in trans_dic_fromApi.keys() },
#         'labels': {I :OldTransDF_idsasKeys['label'][I] for I in trans_dic_fromApi.keys() } #extract labels from old df
   #                      }
#
##check length of the transcript with same Id to see if it complies with lenlist with same id:
#len ( a_meta_dict_transFromApi['transcripts']['-5RCmu-HuTg'].split() )

In [6]:
print(Transcripts_w_timestamps1[1])

['iymP2jq6nFs', 'dHQQm_sJHDA']


In [8]:
def TranscriptExtractor():
    "This Function extracts a list from the dataset with the following string contents: [Video_ID, Video_Category, Video_Transcript, Video_Rating]"
    import os
    import csv
    import json
    # first we load in all info from the youtube.csv we need
   # os.chdir(directory_youtubecsv)
    with open('autumn 2021.csv', mode='r') as infile:
        reader = csv.reader(infile)
        youtube_ranked = []
        for rows in reader:
            youtube_ranked.append(rows)
        youtube_ranked_data = {}
    for listed in youtube_ranked:
        youtube_ranked_data.update({listed[0]: listed[3]})

    with open('ourWatchedYoutubevidsTranscriptsψkeys.json') as f:
        transcripts = json.load(f)
    print(type(youtube_ranked))
    id_cat_transcripts = []
    id_transcripts = list(transcripts.keys())
    for id in id_transcripts:
        id_cat_transcripts.append([id, transcripts[id], youtube_ranked_data[id]])

    return id_cat_transcripts

In [11]:
transcripts_labels = TranscriptExtractor()

import csv
with open('data_autumn2021.csv', 'w', newline='', encoding = 'utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["id", "transcript", "rating"])
with open("data_autumn2021.csv", "a", newline="", encoding = 'utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(transcripts_labels)

<class 'list'>
